<a href="https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding=utf-8
from re import findall
from time import time
from google.colab import output
from IPython.display import display,HTML
try:
	from selenium import webdriver
except:
	# 清除輸出Log
	print('Install package...\n下載所需插件以便執行...')
	!apt -q -qq update
	!apt -q -qq install chromium-chromedriver
	!pip -q install selenium
	from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
# options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#		 INFO = 0, 
#		 WARNING = 1, 
#		 LOG_ERROR = 2, 
#		 LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
# 清除輸出内容
output.clear()
#@markdown ←點擊▶按鈕開始運行刷分\
#@markdown 請先在在下方輸入個人賬號及密碼
username = "tiaccwhf.5a29" #@param {type:"string"}
password = "2020" #@param {type:"string"}
#@markdown 選擇相應課程類型
lessons_type = "Wiseman(EB level 1)" #@param ["Wiseman(EB level 1)", "Fun and Friends(Foundation)"]
#@markdown 設定每課程的分數
score = 100 #@param {type:"slider", min:0, max:100, step:1}
#@markdown 設定每課程的運作時間(單位:秒)\
#@markdown 防止提交過快遭到服務器屏蔽
running_time = 10 #@param {type:"slider", min:10, max:300, step:1}
if lessons_type == "Fun and Friends(Foundation)":
	print('登入Fun and Friends',end='\r')
	driver.get('https://fnfclub.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	print('進入課程列表',end='\r')
	driver.get('https://fnfclub.hk/lms/user/secure/course/selector/do/lessons.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="effect_session"]/table[@class="list_line"]/tbody/tr')))
	display(HTML(
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')+
		driver.find_element_by_xpath('//div[@class="effect_session"]').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')
	))
	url=[]
	for tr in driver.find_elements_by_xpath('//div[@class="effect_session"]/table[@class="list_line"]/tbody/tr'):
		if '-' in tr.find_element_by_xpath('.//td[4]').get_attribute('innerText'):
			try:
				href=tr.find_element_by_xpath('.//td[3]/span/a[contains(@href,"/lms/user/secure/course/selector/do/selectLesson.do?id=")]').get_attribute('href')
				url.append(href)
				print(href,end='\r')
			except NoSuchElementException as identifier:
				print('Next',end='\r')
	url_total=int(len(url))
	print('預計完成時間:%d\'s'%(running_time*url_total))
	url_now=1
	for url in url:
		driver.get(url)
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		token = str(driver.execute_script('return window.location.href;')).replace('https://fnfclub.hk/lms/scorm/?token=','')
		lesson_id = findall(r'id=.*&from=lessons',url)[0].replace('id=','').replace('&from=lessons','')
		print('當前進度:[%d/%d]\ntoken:%s\nlesson_id:%s'%(url_now,url_total,token,lesson_id))
		url_now+=1
		tm=time()
		while time()-tm<=running_time:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
		driver.get('https://fnfclub.hk/lms/scorm/commit.do?token=%s&data={"wiseman.lesson_id":"%s","cmi.core.lesson_status":"completed","wiseman.playground_score":%d,"wiseman_init":"true","cmi.suspend_data":"true","cmi.core.score.raw":%d,"wiseman.hostname":"fnfclub.hk","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":"","wiseman.like":"","wiseman.error_rate":"","wiseman.target":"","wiseman.total_like":""}'%(token,lesson_id,score,score))
		print('Submited',end='\r')
	print('進入課程列表',end='\r')
	driver.get('https://fnfclub.hk/lms/user/secure/course/selector/do/lessons.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="effect_session"]/table[@class="list_line"]/tbody/tr')))
	display(HTML(
		driver.find_element_by_xpath('//div[@class="effect_session"]/div[@class="navigator_padding start"]').get_attribute('innerHTML')+
		'<h1>All Finished</h1>'
	))
	# 登出Fun and Friends
	driver.get('https://fnfclub.hk/lms/user/logout.do')
elif lessons_type == "Wiseman(EB level 1)":
	print('登入Wiseman',end='\r')
	driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	display(HTML(
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+
		driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
		driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]').get_attribute('innerHTML')#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
	))
	print('進入課程列表',end='\r')
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')))
	display(HTML(driver.find_element_by_xpath('//div[@class="table-responsive"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')))
	url=[]
	for tr in driver.find_elements_by_xpath('//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr'):
		if '-' in tr.find_element_by_xpath('.//td[@class="align-middle text-center"]').get_attribute('innerText'):
			try:
				href=tr.find_element_by_xpath('.//a[contains(@href,"/lms/user/secure/course/eb/select_theme/selectLesson")]').get_attribute('href')
				url.append(href)
				print(href,end='\r')
			except NoSuchElementException as identifier:
				print('Next',end='\r')
	url_total=int(len(url))
	print('預計完成時間:%d\'s'%((running_time)*url_total))
	url_now=1
	for url in url:
		driver.get(url)
		wait.until(EC.presence_of_element_located((By.XPATH,'//iframe[@id="course"]')))
		token = str(driver.execute_script('return window.location.href;')).replace('https://lms.wiseman.com.hk/lms/scorm/?token=','')
		print('當前進度:[%d/%d]\ntoken:%s'%(url_now,url_total,token))
		url_now+=1
		tm=time()
		while time()-tm<=running_time:
			print('Waiting:%f\'s'%(time()-tm),end='\r')
		# driver.execute_script('window.location.href=\'https://lms.wiseman.com.hk/lms/scorm/commit.do?token=%s&data={"cmi.core.lesson_status":"completed","wiseman.playground_score":"0","wiseman.like":"0","wiseman_init":"true","wiseman.error_rate":"","wiseman.target":"-1","cmi.suspend_data":"true","cmi.core.score.raw":100,"wiseman.total_like":"0","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":""}\';'%(token))
		driver.get('https://lms.wiseman.com.hk/lms/scorm/commit.do?token=%s&data={"cmi.core.lesson_status":"completed","wiseman.playground_score":"0","wiseman.like":"0","wiseman_init":"true","wiseman.error_rate":"","wiseman.target":"-1","cmi.suspend_data":"true","cmi.core.score.raw":%d,"wiseman.total_like":"0","wiseman.extension":"true","cmi.core.session_time":"","wiseman.category_score":"","wiseman.lesson_type":""}'%(token,score))
		print('Submited',end='\r')
	# 進入分數頁面
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/dashboard.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card-body"]/div[@class="text-right"]')))
	display(HTML(
		driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
		driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]').get_attribute('innerHTML')+#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
		'<h1>All Finished</h1>'
	))
	# 登出Wiseman
	driver.get('https://lms.wiseman.com.hk/lms/user/logout.do')
driver.quit()